In [ ]:
!pip install undetected-chromedriver --quiet
!pip install selenium --quiet

#используем версию webdriver, которая необнаруживается анти-ботами
import undetected_chromedriver as webdriver
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
#загрузим браузер Chrome
driver = webdriver.Chrome()
#создаем запрос get к необходимому веб-ресурсу
driver.get("https://tproger.ru/quiz/real-programmer")
#ожидаем полной загрузки сайта
driver.set_page_load_timeout(10)

#выведем на экран название теста
title = driver.title
print(title, end ="\n\n")

#дождемся появления кнопки "Отправить coockie", и нажмем её чтобы не мешала
button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[3]/button/div/span')))
button.click()

#установим начальный уровень прогресса ответов на тест
progress = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div/div[1]/div/div[1]/div/span')))
progress = driver.find_element(By.CLASS_NAME, 'tp-quiz-progress__counter').text
#time.sleep(5)

#пока уровень прогресса не максимальный используем цикл
while int(progress.split()[0]) <= int(progress.split()[2]):
    #номер текущего вопроса теста
    question_number = int(progress.split()[0])
    #общее количество вопросов теста
    all_question_number = int(progress.split()[2])
    #выводим на экран номер вопроса теста, текущий уровень прогресса
    print(f'Вопрос № {question_number} из {all_question_number}')

    #выводим на экран текст вопроса теста
    question = driver.find_element(By.CLASS_NAME, 'tp-quiz-question__text').text
    print(question)

    #рандомно кликаем на один из вопросов теста
    #для начала определяем номер вопроса, т.к. у первого вопроса иной "адрес" в html-коде изучаемого ресурса
    if int(progress.split()[0]) == 1:
        #ожидаем, когда кнопка станет кликабельной
        answer = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div[2]/div[3]/ul/li[1]/div/label/input")))
    else:
        answer = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div/div[3]/ul/li[1]/div/label/input")))
    #нажимаем на кнопку ответа на вопрос
    answer.click()

    print('Ответы:', end = "\n")
    #спасрсим текст вопросов и процентное количество выбранных ответов
    answers = driver.find_elements(By.CLASS_NAME, "tp-answer__text")
    ans = []
    for i in answers:
        ans.append(i.text)

    percent = driver.find_elements(By.CLASS_NAME, 'tp-answer__percentage')
    per = []
    for v in percent:
        per.append(v.text)

    ans_dict = dict(zip(ans,per))

    for i in range(0,len(ans_dict),1):
        print(list(ans_dict.keys())[i]," : ", list(ans_dict.values())[i])

    #спасрсим и выведем на экран правильный ответ
    if int(progress.split()[0]) == 1:
        right_ans = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div[2]/div[3]/ul/li[1]/div/label/div/span[1]')
    else:
        right_ans = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div/div[3]/ul/li[2]/div/label/div/span[1]')
    text_right_answer = right_ans.text
    print(f'Правильный ответ: {text_right_answer}', end ="\n\n")

    #найдем на странице и нажмем на кнопку "Далее"
    #дождемся, когда кнопка станет кликабельной и нажмем её
    button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div/div[3]/div[2]/button/div/span')))
    button.click()
    #установим время ожидания обновления страницы
    time.sleep(3)

    #если вопросы закончились, заканчиваем выполнение программы, если нет, продолжаем
    if question_number != all_question_number:
        #обновим значение переменной прогресса
        progress = driver.find_element(By.XPATH, '/html/body/div[5]/div/div/main/div[2]/div[1]/div[4]/div/div/div/div/div/div[1]/div/div[1]/div/span').text
    else:
        print('Конец программы.')
        break

driver.close()


Тест: а вы точно программист?

Вопрос № 1 из 12
Начнём с простого. Какую первую программу обычно пишут программисты?
Ответы:
Hello world  :  91%
Сортировку «пузырьком»  :  3%
Для взлома аккаунта «ВКонтакте». Такая программа есть у каждого программиста  :  2%
Это вопрос с подвохом. Начинают они с починки утюгов, программы тут не при чём.  :  4%
Правильный ответ: Hello world

Вопрос № 2 из 12
Бывает ли так, что программа скомпилировалась с первого раза и без ошибок?
Ответы:
Да, конечно  :  29%
Нет, это фантастика  :  71%
Правильный ответ: Нет, это фантастика

Вопрос № 3 из 12
Представим гипотетическую ситуацию, в которой программа всё-таки скомпилировалась с первого раза. Как вы поступите?
Ответы:
Выключу комп и спокойно пойду спать — дело сделано  :  8%
Порадуюсь за себя и продолжу писать код  :  19%
Буду искать ошибку в компиляторе, где-то она должна быть  :  73%
Правильный ответ: Порадуюсь за себя и продолжу писать код

Вопрос № 4 из 12
Движемся дальше. Вопрос на проверку логического 